In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks


YOLOv5 🚀 v7.0-361-gc5ffbbf1 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5845.9/8062.4 GB disk)


In [2]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip


100%|██████████| 780M/780M [00:53<00:00, 15.4MB/s]


In [3]:
# Validate YOLOv5n on COCO val
!python val.py --weights yolov5n.pt --data coco.yaml --img 640 --device 0 --task val


val: data=/kaggle/working/yolov5/data/coco.yaml, weights=['yolov5n.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-361-gc5ffbbf1 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

100%|███████████████████████████████████████| 3.87M/3.87M [00:00<00:00, 218MB/s]

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
val: Scanning /kaggle/working/datasets/coco/val2017... 4952 images, 48 backgroun
val: New cache created: /kaggle/working/datasets/coco/val2017.cache
                 Class     Images  Instances          P          R      mAP50   
                   all       5000      36335      0.574      0.432      0.456      0.277
Speed: 0.1ms pre-process, 1.7ms inference, 2.2ms NMS p

# **ONNX FP32**

In [4]:
# convert pytorch model to onnx fp32 model
!python export.py --weights /kaggle/working/yolov5/yolov5n.pt --img 640 --batch 1 --include onnx
!mv /kaggle/working/yolov5/yolov5n.onnx /kaggle/working/yolov5/yolov5n_fp32.onnx


export: data=data/coco128.yaml, weights=['/kaggle/working/yolov5/yolov5n.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-361-gc5ffbbf1 Python-3.10.13 torch-2.1.2 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs

PyTorch: starting from /kaggle/working/yolov5/yolov5n.pt with output shape (1, 25200, 85) (3.9 MB)

ONNX: starting export with onnx 1.15.0...
ONNX: export success ✅ 0.8s, saved as /kaggle/working/yolov5/yolov5n.onnx (7.6 MB)

Export complete (1.3s)
Results saved to /kaggle/working/yolov5
Detect:          python detect.py --weights /kaggle/working/yolov5/yolov5n.onnx 
Validate:        python val.py --weights /kaggle/working/yolov5/yolov5n.onn

In [5]:
# Validation yolov5n.onnx on COCO val
!python val.py --weights /kaggle/working/yolov5/yolov5n_fp32.onnx --data coco.yaml --img 640 --device 0 --task val


val: data=/kaggle/working/yolov5/data/coco.yaml, weights=['/kaggle/working/yolov5/yolov5n_fp32.onnx'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-361-gc5ffbbf1 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

Loading /kaggle/working/yolov5/yolov5n_fp32.onnx for ONNX Runtime inference...
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 MB 181.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 171.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 218.1 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 18.5s, installed 1 package: ['onnxruntime-gpu']
requirements: ⚠️ R

In [6]:
!pip install onnxruntime
!pip install timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 85.8 MB/s eta 0:00:00


In [7]:
# Quantization Class
import onnxruntime as ort
from onnxruntime import quantization
import timm

class OnnxStaticQuantization:
    def __init__(self) -> None:
        self.enum_data = None
        self.calibration_technique = {
            "MinMax": ort.quantization.calibrate.CalibrationMethod.MinMax,
            "Entropy": ort.quantization.calibrate.CalibrationMethod.Entropy,
            "Percentile": ort.quantization.calibrate.CalibrationMethod.Percentile,
            "Distribution": ort.quantization.calibrate.CalibrationMethod.Distribution
        }
        
    def get_next(self, EP_list = ['CPUExecutionProvider']):
        if self.enum_data is None:
            session = ort.InferenceSession(self.fp32_onnx_path, providers=EP_list)
            input_name = session.get_inputs()[0].name
            calib_list = []
            count = 0
            for nhwc_data, _ in self.calibration_loader:
                nhwc_data=nhwc_data.cpu()
                calib_list.append({input_name: nhwc_data.numpy()})
                if self.sample == count: break
                count = count + 1
            self.enum_data = iter(calib_list)
        return next(self.enum_data, None)
    
    def quantization(self, fp32_onnx_path, future_int8_onnx_path, calib_method, calibration_loader, sample=100):
        self.sample = sample
        self.calibration_loader = calibration_loader
        _ = ort.quantization.quantize_static(
                model_input=fp32_onnx_path,
                model_output=future_int8_onnx_path,
                activation_type=ort.quantization.QuantType.QInt16, 
                weight_type=ort.quantization.QuantType.QInt16,
                calibrate_method=self.calibration_technique[calib_method],
                calibration_data_reader=self
            )
        return self




AttributeError: INT4

In [ ]:
# ====================================================================================
# Perform the quantization
val_dataset = timm.data.ImageDataset('/kaggle/working/datasets/coco/images/val2017')
val_loader = timm.data.create_loader(val_dataset, (1,3,640,640), 1)
module = OnnxStaticQuantization()
module.fp32_onnx_path = "/kaggle/working/yolov5/yolov5n_fp32.onnx"

# method=Distribution, calibration_number=100
module.quantization(
    fp32_onnx_path="yolov5n_fp32.onnx",
    future_int8_onnx_path="yolov5n_fp32_MinMax.onnx",
    calib_method="MinMax",
    calibration_loader=val_loader,
    sample=1000   # calibration number
)



In [ ]:
# Validation yolov5n_fp32_MinMax.onnx on COCO val
!python val.py --weights /kaggle/working/yolov5/yolov5n_fp32_MinMax.onnx --data coco.yaml --img 640 --device 0 --task val


## **Different Calibration Methods**

In [ ]:
# Quantization Class
import onnxruntime as ort
from onnxruntime import quantization
import timm

class OnnxStaticQuantization:
    def __init__(self) -> None:
        self.enum_data = None
        self.calibration_technique = {
            "MinMax": ort.quantization.calibrate.CalibrationMethod.MinMax,
            "Entropy": ort.quantization.calibrate.CalibrationMethod.Entropy,
            "Percentile": ort.quantization.calibrate.CalibrationMethod.Percentile,
            "Distribution": ort.quantization.calibrate.CalibrationMethod.Distribution
        }
    def get_next(self, EP_list = ['CPUExecutionProvider']):
        if self.enum_data is None:
            session = ort.InferenceSession(self.fp32_onnx_path, providers=EP_list)
            input_name = session.get_inputs()[0].name
            calib_list = []
            count = 0
            for nhwc_data, _ in self.calibration_loader:
                nhwc_data=nhwc_data.cpu()
                calib_list.append({input_name: nhwc_data.numpy()})
                if self.sample == count: break
                count = count + 1
            self.enum_data = iter(calib_list)
        return next(self.enum_data, None)
    def quantization(self, fp32_onnx_path, future_int8_onnx_path, calib_method, calibration_loader, sample=100):
        self.sample = sample
        self.calibration_loader = calibration_loader
        _ = ort.quantization.quantize_static(
                model_input=fp32_onnx_path,
                model_output=future_int8_onnx_path,
                activation_type=ort.quantization.QuantType.QInt16, weight_type=ort.quantization.QuantType.QInt8,
                calibrate_method=self.calibration_technique[calib_method],
                calibration_data_reader=self
            )
        return self



In [ ]:
# ====================================================================================
# Perform the quantization
val_dataset = timm.data.ImageDataset('/kaggle/working/datasets/coco/images/val2017')
val_loader = timm.data.create_loader(val_dataset, (1,3,640,640), 1)
module = OnnxStaticQuantization()
module.fp32_onnx_path = "/kaggle/working/yolov5/yolov5n_fp32.onnx"

# method=Entropy, calibration_number=1000
module.quantization(
    fp32_onnx_path="/kaggle/working/yolov5/yolov5n_fp32.onnx",
    future_int8_onnx_path="/kaggle/working/yolov5/yolov5n_fp32_Entropy.onnx",
    calib_method="Entropy",
    calibration_loader=val_loader,
    sample=100   # calibration number
)


In [ ]:
# Validation yolov5n_fp32_int8.onnx on COCO val
!python val.py --weights /kaggle/working/yolov5/yolov5n_fp32_Entropy.onnx --data coco.yaml --img 640 --device 0 --task val

In [ ]:
# Quantization Class
import onnxruntime as ort
from onnxruntime import quantization
import timm

class OnnxStaticQuantization:
    def __init__(self) -> None:
        self.enum_data = None
        self.calibration_technique = {
            "MinMax": ort.quantization.calibrate.CalibrationMethod.MinMax,
            "Entropy": ort.quantization.calibrate.CalibrationMethod.Entropy,
            "Percentile": ort.quantization.calibrate.CalibrationMethod.Percentile,
            "Distribution": ort.quantization.calibrate.CalibrationMethod.Distribution
        }
    def get_next(self, EP_list = ['CPUExecutionProvider']):
        if self.enum_data is None:
            session = ort.InferenceSession(self.fp32_onnx_path, providers=EP_list)
            input_name = session.get_inputs()[0].name
            calib_list = []
            count = 0
            for nhwc_data, _ in self.calibration_loader:
                nhwc_data=nhwc_data.cpu()
                calib_list.append({input_name: nhwc_data.numpy()})
                if self.sample == count: break
                count = count + 1
            self.enum_data = iter(calib_list)
        return next(self.enum_data, None)
    def quantization(self, fp32_onnx_path, future_int8_onnx_path, calib_method, calibration_loader, sample=100):
        self.sample = sample
        self.calibration_loader = calibration_loader
        _ = ort.quantization.quantize_static(
                model_input=fp32_onnx_path,
                model_output=future_int8_onnx_path,
                activation_type=ort.quantization.QuantType.QInt16, weight_type=ort.quantization.QuantType.QInt8,
                calibrate_method=self.calibration_technique[calib_method],
                calibration_data_reader=self
            )
        return self

 

In [ ]:
   
# ====================================================================================
# Perform the quantization
val_dataset = timm.data.ImageDataset('/kaggle/working/datasets/coco/images/val2017')
val_loader = timm.data.create_loader(val_dataset, (1,3,640,640), 1)
module = OnnxStaticQuantization()
module.fp32_onnx_path = "/kaggle/working/yolov5/yolov5n_fp32.onnx"

# method=Percentile, calibration_number=1000
module.quantization(
    fp32_onnx_path="/kaggle/working/yolov5/yolov5n_fp32.onnx",
    future_int8_onnx_path="/kaggle/working/yolov5/yolov5n_fp32_Percentile.onnx",
    calib_method="Percentile",
    calibration_loader=val_loader,
    sample=500   # calibration number
)


In [ ]:
# Validation yolov5n_fp32_Percentile.onnx on COCO val
!python val.py --weights /kaggle/working/yolov5/yolov5n_fp32_Percentile.onnx --data coco.yaml --img 640 --device 0 --task val

In [ ]:
# Quantization Class
import onnxruntime as ort
from onnxruntime import quantization
import timm

class OnnxStaticQuantization:
    def __init__(self) -> None:
        self.enum_data = None
        self.calibration_technique = {
            "MinMax": ort.quantization.calibrate.CalibrationMethod.MinMax,
            "Entropy": ort.quantization.calibrate.CalibrationMethod.Entropy,
            "Percentile": ort.quantization.calibrate.CalibrationMethod.Percentile,
            "Distribution": ort.quantization.calibrate.CalibrationMethod.Distribution
        }
    def get_next(self, EP_list = ['CPUExecutionProvider']):
        if self.enum_data is None:
            session = ort.InferenceSession(self.fp32_onnx_path, providers=EP_list)
            input_name = session.get_inputs()[0].name
            calib_list = []
            count = 0
            for nhwc_data, _ in self.calibration_loader:
                nhwc_data=nhwc_data.cpu()
                calib_list.append({input_name: nhwc_data.numpy()})
                if self.sample == count: break
                count = count + 1
            self.enum_data = iter(calib_list)
        return next(self.enum_data, None)
    def quantization(self, fp32_onnx_path, future_int8_onnx_path, calib_method, calibration_loader, sample=100):
        self.sample = sample
        self.calibration_loader = calibration_loader
        _ = ort.quantization.quantize_static(
                model_input=fp32_onnx_path,
                model_output=future_int8_onnx_path,
                activation_type=ort.quantization.QuantType.QInt16, weight_type=ort.quantization.QuantType.QInt8,
                calibrate_method=self.calibration_technique[calib_method],
                calibration_data_reader=self
            )
        return self

    
# ====================================================================================
# Perform the quantization
val_dataset = timm.data.ImageDataset('/kaggle/working/datasets/coco/images/val2017')
val_loader = timm.data.create_loader(val_dataset, (1,3,640,640), 1)
module = OnnxStaticQuantization()
module.fp32_onnx_path = "/kaggle/working/yolov5/yolov5n_fp32.onnx"

# method=Distribution, calibration_number=5000
# Perform the quantization
val_dataset = timm.data.ImageDataset('/kaggle/working/datasets/coco/images/val2017')
val_loader = timm.data.create_loader(val_dataset, (1,3,640,640), 1)
module = OnnxStaticQuantization()
module.fp32_onnx_path = "/kaggle/working/yolov5/yolov5n_fp32.onnx"

module.quantization(
    fp32_onnx_path="/kaggle/working/yolov5/yolov5n_fp32.onnx",
    future_int8_onnx_path="/kaggle/working/yolov5/yolov5n_fp32_Distribution.onnx",
    calib_method="Distribution",
    calibration_loader=val_loader,
    sample=1000   # calibration number
)

In [ ]:
# Validation yolov5n_fp32_int8.onnx on COCO val
!python val.py --weights /kaggle/working/yolov5/yolov5n_fp32_Distribution.onnx --data coco.yaml --img 640 --device 0 --task val

## **TVM**

In [ ]:
# TVM : Tensor Virtual Machine

import torchvision.datasets as datasets
from torch.utils.data import Subset, DataLoader
import torchvision.transforms as transforms
import onnx
import tvm
import tvm.relay as relay
import numpy as np
import time
from tqdm import tqdm


# Import the data_folder and transform variables
data_folder = '/path/to/your/data/folder'
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Test Dataset
test_dataset = datasets.ImageFolder(root=data_folder, transform=transform)

# Create a DataLoader for the test dataset
subset_indices = list(range(0, len(test_dataset), 50))
subset_dataset = Subset(test_dataset, subset_indices)
test_loader = DataLoader(subset_dataset, batch_size=1, shuffle=False)

# Tensor Virtual Machine
def eval_tvm(args, data_loader):
    input_shape = (1, 3, 640, 640)
    model_path = args.model + "_int8" + ".onnx"
    onnx_model = onnx.load(model_path)
    input_name = "input"
    shape_dict = {input_name: input_shape}
    mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)
    target = "llvm"
    with tvm.transform.PassContext(opt_level=3):
        executor = relay.build_module.create_executor(
            "graph", mod, tvm.cpu(0), target, params
        ).evaluate()

    print("Finished TVM conversion")

    top1_correct = 0
    top5_correct = 0
    total_samples = 0
    start_time = time.time()
    for idx, (images, labels) in enumerate(data_loader):
        # Set the input data
        numpy_images = images.numpy()
        input_data = tvm.nd.array(numpy_images.astype("float32"))
        tvm_output = executor(input_data).numpy()
        predicted_labels = np.argmax(tvm_output, axis=1)
        top1_correct += np.sum(predicted_labels == labels.numpy())

        # Calculate top-5 accuracy
        top5_predicted_labels = np.argsort(tvm_output, axis=1)[:, -5:]
        for i in range(labels.size(0)):
            if labels.numpy()[i] in top5_predicted_labels[i]:
                top5_correct += 1

        total_samples += labels.size(0)

        if idx >= 999:  # Process 1000 batches
            break

    end_time = time.time()
    inference_time = end_time - start_time
    print("Inference Time:", inference_time, "seconds")

    # Calculate accuracy
    top1_accuracy = top1_correct / total_samples
    top5_accuracy = top5_correct / total_samples

    print(f"Top-1 Accuracy: {top1_accuracy * 100:.2f}%")
    print(f"Top-5 Accuracy: {top5_accuracy * 100:.2f}%")


# Example usage
args = {'model': '/path/to/your/model'}
eval_tvm(args, test_loader)


